---
title: Informes Cambio de Componentes
subtitle: Revisión de completitud
author: Chief Reliability
date: 2023-01-23
---

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path


import pandas as pd
import polars as pl
import dagster as dg
from kdags.resources import *
from io import BytesIO
import pdfplumber
from datetime import datetime
import re
import os
from kdags.definitions import *
from kdags.readr import Readr
from kdags.resources.ggplot import *
from kdags.resources.tidyr import *

import matplotlib.pyplot as plt
import numpy as np

# from itables import init_notebook_mode, show
from datetime import date

# init_notebook_mode(all_interactive=True)

context = dg.build_asset_context()


def generate_weekly_summary(df):

    weekly_summary = df.group_by("changeout_week").agg(
        pl.col("changeout_date").count().alias("total"),
        (~pl.col("has_report")).cast(pl.Int64).sum().alias("missing"),
    )

    # Calculate percentages and available counts
    weekly_summary = weekly_summary.with_columns(
        (pl.col("missing") / pl.col("total") * 100)
        .round(1)
        .alias("missing_percentage"),
        (pl.col("total") - pl.col("missing")).alias("available"),
    )

    # Sort by week to ensure chronological order
    weekly_summary = weekly_summary.sort("changeout_week", descending=True)

    # Filter to weeks with at least one record
    weekly_summary = weekly_summary.filter(pl.col("total") > 0)

    return weekly_summary


def plot_icc_report(weekly_summary):
    """
    Create the ICC report visualization using GGPrism.

    Parameters:
        weekly_summary (pd.DataFrame): Weekly summary from generate_weekly_summary function

    Returns:
        tuple: (fig, ax) - Matplotlib figure and axis objects
    """
    # Create theme and figure
    theme = GGPrism()
    fig, ax = theme.create_figure(figsize=(12, 7))

    # Plot stacked bars
    x = np.arange(len(weekly_summary))
    ax.bar(
        x,
        weekly_summary["available"],
        width=0.85,
        color=theme.COLORS[0],
        label="Available",
        edgecolor="white",
        linewidth=0.8,
    )
    ax.bar(
        x,
        weekly_summary["missing"],
        width=0.85,
        bottom=weekly_summary["available"],
        color=theme.COLORS[4],
        label="Missing",
        edgecolor="white",
        linewidth=0.8,
    )

    # Add percentage labels with improved positioning
    for i, row in enumerate(weekly_summary.itertuples()):
        # Adjust vertical position based on total value to prevent overlap
        vertical_offset = 0.3  # Base offset

        # Add more space for higher bars to prevent label overlap
        if row.total >= 5:
            vertical_offset = 0.4

        if row.missing > 0:
            ax.text(
                i,
                row.total + vertical_offset,
                f"{int(row.missing_percentage)}%",
                ha="center",
                va="bottom",
                fontsize=13,
                fontweight="bold",
                color=theme.FIXED_COLORS["axis_color"],
            )
        else:
            ax.text(
                i,
                row.total + vertical_offset,
                "0%",
                ha="center",
                va="bottom",
                fontsize=13,
                fontweight="bold",
                color=theme.FIXED_COLORS["axis_color"],
            )

    # Set x-ticks
    ax.set_xticks(x)
    ax.set_xticklabels(weekly_summary["week"], rotation=90)

    # Ensure y-axis has enough room for labels
    y_max = max(weekly_summary["total"]) + 1.5  # Add extra space for labels
    ax.set_ylim(0, y_max)

    # Apply styling and finalize plot
    theme.style_legend(ax, title="Report Status", loc="upper right")
    theme.finalize_plot(
        fig,
        ax,
        title="Weekly ICC Reports Status",
        xlabel="Week",
        ylabel="Number of Reports",
    )

    return fig


result = kdefs.get_job_def("icc_job").execute_in_process(raise_on_error=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - RUN_START - Started execution of run for "icc_job".
2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - ENGINE_EVENT - Executing steps in process (pid: 23100)
2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - RESOURCE_INIT_STARTED - Starting initialization of resources [io_manager].
2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - RESOURCE_INIT_SUCCESS - Finished initialization of resources [io_manager].
2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - LOGS_CAPTURED - Started capturing logs in process (pid: 23100).
2025-04-10 10:01:10 -0400 - dagster - DEBUG - icc_job - 38324344-379c-4e58-ade8-6aa348ca8e0c - 23100 - gather_icc_reports - STEP_START - Started execution of step "

In [ ]:
# result = kdefs.get_job_def("icc_job").execute_in_process(raise_on_error=True)

In [ ]:
# kdefs.get_job_def("scrape_component_status_job").execute_in_process(raise_on_error=True)

In [2]:
df = result.asset_value("icc").clone()
df

equipment_name,equipment_hours,equipment_hours_icc,component_name,component_code,position_name,position_code,report_date,changeout_date,customer_work_order,filename,file_type,failure_description,iso_year,iso_week,shift
str,i64,f64,str,str,str,i64,datetime[μs],date,i64,str,str,str,i32,i8,str
null,null,74.719,null,null,null,null,2024-12-10 00:00:00,null,null,"""ICC TK399 ST1 2024-12-08""","""pdf""",null,null,null,null
"""TK857""",74693,null,"""modulo_potencia""","""MP""","""unico""",0,null,2025-04-07,-1,null,null,"""Se procede a cambio de modulo …",2025,15,"""N"""
"""TK862""",65500,65500.0,"""cilindro_direccion""","""CD""","""izquierdo""",1,2025-04-07 00:00:00,2025-04-06,-1,"""ICC TK862 CD1 2025-04-06""","""pdf""","""Se procede a cambio de cilindr…",2025,14,"""N"""
"""TK859""",66539,66539.0,"""cilindro_direccion""","""CD""","""derecho""",2,2025-04-06 00:00:00,2025-04-04,-1,"""ICC TK859 CD2 2025-04-04""","""pdf""","""Se procede a cambio de cilindr…",2025,14,"""N"""
"""TK857""",74693,74693.0,"""blower_parrilla""","""BP""","""derecho""",2,2025-04-07 00:00:00,2025-04-03,-1,"""ICC TK857 BP2 2025-04-03""","""pdf""","""Se procede a cambio de blower …",2025,14,"""N"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TK291""",26067,26.067,"""blower_parrilla""","""BP""","""derecho""",2,2024-10-08 00:00:00,2024-10-03,-1,"""ICC TK291 BP2 2024-10-03""","""pdf""","""Se procede a cambio de blower …",2024,40,"""N"""
"""TK855""",67049,67.049,"""blower_parrilla""","""BP""","""derecho""",2,2024-10-01 00:00:00,2024-09-30,-1,"""ICC TK855 BP2 2024-09-30""","""pdf""","""Se procede a cambio de Blower …",2024,40,"""M"""
"""TK867""",62824,62.824,"""suspension_trasera""","""ST""","""derecho""",2,2024-09-29 00:00:00,2024-09-28,18129130,"""ICC TK867 ST2 2024-09-28""","""pdf""","""Se procede a cambio de suspens…",2024,39,"""M"""


In [3]:
df.filter(
    (pl.col("equipment_name") == "TK399")
    & (pl.col("component_name") == "suspension_trasera")
).sort("changeout_date", descending=True)

equipment_name,equipment_hours,equipment_hours_icc,component_name,component_code,position_name,position_code,report_date,changeout_date,customer_work_order,filename,file_type,failure_description,iso_year,iso_week,shift
str,i64,f64,str,str,str,i64,datetime[μs],date,i64,str,str,str,i32,i8,str
"""TK399""",74719,null,"""suspension_trasera""","""ST""","""derecho""",2,null,2024-12-08,-1,null,null,"""REGULARIZACIÓN DE POSICIÓN EN …",2024,49,"""M"""


In [ ]:
icc_df = (
    Readr.Reliability.read_icc(context)
    .with_columns(
        has_report=pl.col("report_date").is_not_null(),
        changeout_week=pl.col("changeout_date").dt.strftime("%Y-W%V"),
    )
    .sort("changeout_date", descending=True)
)
cc_df = Readr.Planning.read_cc().sort("changeout_date", descending=True)
weekly_summary_df = generate_weekly_summary(icc_df)

In [ ]:
df = cc_df.clone().to_pandas()
df = df.loc[df["subcomponent_name"] == "transmision"]
# 1. Ensure 'changeout_date' is in datetime format
df["changeout_date"] = pd.to_datetime(df["changeout_date"])

# 2. Extract the year from the changeout date
df["year"] = df["changeout_date"].dt.year
df.to_csv("components.csv")

In [ ]:
# .to_csv("components.csv")

In [ ]:
# 3. Calculate the total number of changeouts per year
yearly_totals = df.groupby("year").size()

# 4. Filter for changeouts where component_usage is less than 0.8
filtered_df = df[
    df["component_usage"] < 0.8
].copy()  # Use .copy() to avoid SettingWithCopyWarning

# 5. Calculate the number of filtered changeouts per year
yearly_filtered_counts = filtered_df.groupby("year").size()

# 6. Combine the counts into a single DataFrame for easier calculation
summary_df = pd.DataFrame(
    {"total_changeouts": yearly_totals, "filtered_changeouts": yearly_filtered_counts}
)

# Fill NaN with 0 (in case a year has totals but no filtered counts)
summary_df["filtered_changeouts"] = summary_df["filtered_changeouts"].fillna(0)

# 7. Calculate the proportion (handle potential division by zero if a year had 0 changeouts)
summary_df["proportion"] = summary_df.apply(
    lambda row: (
        row["filtered_changeouts"] / row["total_changeouts"]
        if row["total_changeouts"] > 0
        else 0
    ),
    axis=1,
)

# --- Plotting ---

# Create the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Create the bar plot
summary_df["proportion"].plot(kind="bar", ax=ax, color="skyblue")

# Format the y-axis as percentage
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

# Set labels and title
ax.set_xlabel("Year")
ax.set_ylabel("Proportion of Changeouts (Usage < 0.8)")
ax.set_title("Proportion of Low Usage (<0.8) Changeouts per Year")
ax.tick_params(axis="x", rotation=45)  # Rotate x-axis labels if needed

# Add grid lines for better readability
ax.grid(axis="y", linestyle="--", alpha=0.7)

# Ensure layout is tight
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
icc_df.to_pandas().head()

In [ ]:
icc_df.to_pandas().head()

In [ ]:
get_shift_dates().filter(pl.col("date").dt.year() >= 2025)

In [ ]:
icc_df.to_pandas()  # .filter(pl.col("equipment_name") == "TK291")

In [ ]:
icc_df.filter(pl.col("equipment_name") == "TK881")

In [ ]:
cc_df.filter(pl.col("equipment_name") == "TK881").filter(
    pl.col("component_name") == "conjunto_masa_suspension"
).sort("changeout_date", descending=True)

In [ ]:
icc_df

In [ ]:
weekly_summary_df

In [ ]:
from kdags.resources.ggplot.core import GGPrism
from kdags.resources.ggplot.basic_plots import bar_plot


def plot_icc_report_alternative(weekly_summary):
    """
    Alternative implementation using the bar_plot function from basic_plots.
    """
    # Create theme instance
    theme = GGPrism()

    # Make sure we're working with a Polars DataFrame
    if not isinstance(weekly_summary, pl.DataFrame):
        weekly_summary = pl.from_pandas(weekly_summary)

    # Create a stacked bar plot
    fig, ax = bar_plot(
        theme=theme,
        df=weekly_summary,
        x="changeout_week",
        y=["available", "missing"],
        figsize=(12, 7),
        stacked=True,
        width=0.85,
    )

    # Get data as numpy arrays for custom annotations
    total = weekly_summary.select("total").to_numpy().flatten()
    missing_percentage = (
        weekly_summary.select("missing_percentage").to_numpy().flatten()
    )

    # Add percentage labels
    for i in range(len(weekly_summary)):
        vertical_offset = 0.3 if total[i] < 5 else 0.4
        percentage_text = (
            f"{int(missing_percentage[i])}%" if missing_percentage[i] > 0 else "0%"
        )

        ax.text(
            i,
            total[i] + vertical_offset,
            percentage_text,
            ha="center",
            va="bottom",
            fontsize=13,
            fontweight="bold",
            color=theme.FIXED_COLORS["axis_color"],
        )

    # Set x-tick rotation
    plt.setp(ax.get_xticklabels(), rotation=90)

    # Ensure y-axis has enough room for labels
    y_max = max(total) + 1.5
    ax.set_ylim(0, y_max)

    # Finalize plot
    theme.finalize_plot(
        fig,
        ax,
        title="Weekly ICC Reports Status",
        xlabel="Week",
        ylabel="Number of Reports",
        legend_title="Report Status",
    )

    return fig, ax


plot_icc_report_alternative(weekly_summary_df)

In [ ]:
icc_df

In [ ]:
icc_df.filter(
    pl.col("filename").str.contains("TK285")
    # & (pl.col("component_name") == "suspension_trasera")
)

In [ ]:
cc_df.filter(
    (pl.col("equipment_name") == "TK285")
    & (pl.col("component_name") == "suspension_trasera")
)  # ICC TK285 ST2 2024-11-12

In [ ]:
from kdags.resources.ggplot import core as ggprism


def plot_icc_report(weekly_summary):
    """
    Create the ICC report visualization with winter bright theme.

    Parameters:
        weekly_summary (pd.DataFrame): Weekly summary DataFrame

    Returns:
        tuple: (fig, ax) - Matplotlib figure and axis objects
    """
    # Create figure with theme
    fig, ax = ggprism.create_figure(figsize=(12, 7))

    # Get data as numpy arrays
    x = np.arange(len(weekly_summary))
    available = weekly_summary["available"].values
    missing = weekly_summary["missing"].values
    total = weekly_summary["total"].values
    missing_percentage = weekly_summary["missing_percentage"].values

    # Plot stacked bars
    ax.bar(
        x,
        available,
        width=0.85,
        color=ggprism.COLORS[0],
        label="Available",
        edgecolor="white",
        linewidth=0.8,
    )
    ax.bar(
        x,
        missing,
        width=0.85,
        bottom=available,
        color=ggprism.COLORS[4],
        label="Missing",
        edgecolor="white",
        linewidth=0.8,
    )

    # Add percentage labels
    for i, row in enumerate(weekly_summary.itertuples()):
        vertical_offset = 0.3
        if row.total >= 5:
            vertical_offset = 0.4

        percentage_text = f"{int(row.missing_percentage)}%" if row.missing > 0 else "0%"
        ax.text(
            i,
            row.total + vertical_offset,
            percentage_text,
            ha="center",
            va="bottom",
            fontsize=13,
            fontweight="bold",
            color=ggprism.AXIS_COLOR,
        )

    # Set x-ticks
    ax.set_xticks(x)
    ax.set_xticklabels(weekly_summary["changeout_week"], rotation=90)

    # Set y-axis limit
    y_max = max(total) + 1.5
    ax.set_ylim(0, y_max)

    # Set labels and title
    ax.set_title(
        "Weekly ICC Reports Status",
        color=ggprism.AXIS_COLOR,
        fontsize=14,
        fontweight="bold",
    )
    ax.set_xlabel("Week", color=ggprism.AXIS_COLOR, fontsize=12, fontweight="bold")
    ax.set_ylabel(
        "Number of Reports", color=ggprism.AXIS_COLOR, fontsize=12, fontweight="bold"
    )

    # Style legend
    ggprism.style_legend(ax, title="Report Status", loc="upper right")

    # Apply tight layout
    plt.tight_layout()

    return fig, ax


plot_icc_report(weekly_summary_df.to_pandas())

In [ ]:
weekly_summary_df = generate_weekly_summary(icc_df)
weekly_summary_df

In [ ]:
icc_df.filter(pl.col("changeout_date").is_null())

## Cantidad de reportes faltantes por semana

In [ ]:
df = clean_icc_data(icc_df)
weekly_summary = generate_weekly_summary(df)
fig = plot_icc_report(weekly_summary)

In [ ]:
df.sort_values("changeout_date", ascending=False)

La semana es de lunes a domingo.

## Cambios de componente faltantes

In [ ]:
missing_icc_df = (
    icc_df.loc[icc_df["has_report"] == False]
    .drop(
        columns=[
            "equipment_hours_icc",
            "position_code",
            "report_date",
            "filename",
            "__index_level_0__",
            "has_report",
        ]
    )
    .sort_values("changeout_date")
)


def find_matching_cc_records(missing_icc_df, cc_df):
    # Convert DataFrames to Polars if they aren't already
    if not isinstance(missing_icc_df, pl.DataFrame):
        missing_icc_df = pl.from_pandas(missing_icc_df)
    if not isinstance(cc_df, pl.DataFrame):
        cc_df = pl.from_pandas(
            cc_df[
                [
                    "equipment_name",
                    "component_name",
                    "subcomponent_name",
                    "position_name",
                    "changeout_date",
                    "component_hours",
                    "removed_component_serial",
                    "installed_component_serial",
                    "equipment_hours",
                    "customer_work_order",
                    "failure_description",
                ]
            ].assign(
                equipment_hours=lambda x: pd.to_numeric(
                    x["equipment_hours"], errors="coerce"
                )
            )
        )

    # Ensure changeout_date is properly formatted

    # Prepare results list
    results = []

    # Process each missing ICC record
    for missing_row in missing_icc_df.iter_rows(named=True):
        # Find matching records in cc_df
        matches = cc_df.filter(
            (pl.col("equipment_name") == missing_row["equipment_name"])
            & (pl.col("component_name") == missing_row["component_name"])
            & (pl.col("position_name") == missing_row["position_name"])
            & (pl.col("changeout_date") == missing_row["changeout_date"])
        )

        if matches.height == 0:
            # No match found
            result = {
                "equipment_name": missing_row["equipment_name"],
                "component_name": missing_row["component_name"],
                "position_name": missing_row["position_name"],
                "changeout_date": missing_row["changeout_date"].strftime("%Y-%m-%d"),
                "matches_found": 0,
                "component_hours": "Missing",
                "removed_component_serial": "Missing",
                "installed_component_serial": "Missing",
                "subcomponent_name": "Missing",
            }
        else:
            # Create a result with match information
            match_info = matches.select(
                [
                    "component_hours",
                    "removed_component_serial",
                    "installed_component_serial",
                    "subcomponent_name",
                    "customer_work_order",
                    "equipment_hours",
                    "failure_description",
                ]
            ).to_dicts()[
                0
            ]  # Take the first match if multiple

            result = {
                "equipment_name": missing_row["equipment_name"],
                "component_name": missing_row["component_name"],
                "position_name": missing_row["position_name"],
                "changeout_date": missing_row["changeout_date"].strftime("%Y-%m-%d"),
                "matches_found": matches.height,
                **match_info,
            }

        results.append(result)

    # Create DataFrame from results
    result_df = pl.DataFrame(results)

    # Format component_hours as integers if they're numeric
    result_df = result_df.with_columns(
        component_hours=pl.col("component_hours").cast(pl.Float64, strict=False)
    )

    return result_df


result_df = find_matching_cc_records(missing_icc_df, cc_df).drop(["matches_found"])
result_df = result_df.join(
    pl.from_pandas(
        MasterData.equipments()[
            ["equipment_name", "equipment_serial", "equipment_model"]
        ]
    ),
    on="equipment_name",
    how="left",
)
show(result_df)

## Reportes sin ser asignados

In [ ]:
icc_files = [
    f
    for f in (
        Path(os.environ["ONEDRIVE_LOCAL_PATH"]) / "INFORMES_CAMBIO_DE_COMPONENTE"
    ).rglob("*")
    if ((f.is_file()) & (f.suffix == ".pdf") & (f.stem.lower().startswith("icc")))
]
icc_files = pl.DataFrame({"filepath": icc_files}).with_columns(
    filename=pl.col("filepath").map_elements(lambda x: x.stem)
)
unamended_icc = icc_files.join(icc_df, how="left", on="filename")
unamended_icc = unamended_icc.filter(pl.col("report_date").is_null())
if unamended_icc.shape[0] == 0:
    print("No unamended ICC reports found")
unamended_icc

In [ ]:
# icc_files
icc_files